## Goal:

This example shows how to calculate the Reynolds stress tensor elements from the 6-beam observations. 

## Steps:

1) Read the WindCube's data
2) Merge data from one hour of observations
3) Re-structure the data for using the wind retrieval
4) Calculate the Reynolds stress tensor elements

In [1]:
import glob as gb
import matplotlib.pyplot as plt

import lidarSuit as lst

### Step 1 and 2: reading and merging the 6-beam data

In [2]:
# a 6-beam sample data is available at the path below
data_path = '../../sampleData/wind_and_aerosols_data/12-00/*.nc'
file_list = sorted(gb.glob(data_path))
mergedDS = lst.dataOperations(file_list).mergedData

Below you can see all the variables available on the original WindCube's data. Not all of them are needed for retrieving the wind profiles. Note that the variables from the vertical observations are kept separate from the slanted observations. 

In [3]:
# Uncomment the line below

# mergedDS

### Step 3: re-structuring the data

In [4]:
restruct_data = lst.getRestructuredData(mergedDS)

The re-structured object contains all the information needed to calculate the Reynolds stress tensor elements.

### Step 4: Calculate the Reynolds stress tensor elements

To calculate Reynolds stress tensor elements, you need to define a time window (a frequency). Since lidarSuit takes advantage of the xarray library, the frequency for calculating those elements is defined in terms of the number of profiles. However, a frequency in terms of time is more practical. In the following fuor lines, a time frequency will be related to its equivalent in terms of profile frequency.

In [5]:
# desired time windown in minutes
time_window = 5

# duration of one minute in seconds
minute_lenght = 60

# vertical observations time resolution in seconds
time_resolution = restruct_data.dataTransf90.time.diff(dim='time').values * 1e-9
time_resolution = int(time_resolution[0])

# frequency convertion from minutes to profile number
freq = (minute_lenght/time_resolution)*time_window
freq = int(freq)

Applying the 6-beam method to calculate Reynolds stress tensor elements

In [6]:
turb_data = lst.sixBeamMethod(restruct_data, freq=freq, freq90=freq)

/Users/jdiasneto/.local/lib/python3.8/site-packages/xarray/core/nputils.py:155: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)
/Users/jdiasneto/.local/lib/python3.8/site-packages/xarray/core/nputils.py:155: RuntimeWarning: Degrees of freedom <= 0 for slice.
  result = getattr(npmodule, name)(values, axis=axis, **kwargs)


Have a look at the Reynolds elements dataset

In [7]:
# Uncomment the line below 

# turb_data.varCompDS

Below we will have the first impressions of the retrieved data. Note that there are negative variances, which is mathematically impossible. Those negative is a known artefact of the 6-beam method.

In [8]:
# Uncomment the lines below to see the plots

# turb_data.varCompDS.var_u.plot(x='time', cmap='Spectral', vmin=-5, vmax=5)
# plt.show()

# turb_data.varCompDS.var_v.plot(x='time', cmap='Spectral', vmin=-5, vmax=5)
# plt.show()

# turb_data.varCompDS.var_w.plot(x='time', cmap='Spectral', vmin=0, vmax=1)
# plt.show()